<a href="https://colab.research.google.com/github/andssuu/moncattle/blob/master/supervised/notebooks/random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/andssuu/moncattle.git

Cloning into 'moncattle'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (308/308), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 308 (delta 145), reused 129 (delta 54), pack-reused 0
Receiving objects: 100% (308/308), 3.18 MiB | 13.31 MiB/s, done.
Resolving deltas: 100% (145/145), done.


# Importação das bibliotecas

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Leitura da base de dados

In [53]:
df = pd.read_csv('moncattle/data/lomba.csv')
data = df[df.columns[1:10]]
data_normalized = (data - data.min()) / (data.max() - data.min())
labels = df[df.columns[-1]]
#le = preprocessing.LabelEncoder()
#le.fit(labels.values)
#labels = le.transform(labels.values)
data.head()

,acc_x,acc_y,acc_z,mag_x,mag_y,mag_z,giro_x,giro_y,giro_z
0,-11889,11184,-17877,-1280,-73,-309,-4013,1260,-31
1,-9144,8945,-11799,-1213,707,-238,-394,2278,-602
2,6391,9265,-11767,570,-405,-1533,-1072,-1094,1854
3,-1394,10206,-13229,671,-73,-1590,287,2253,-2760
4,-4864,5953,-10472,-84,-200,-1767,-7101,9206,-3595


In [54]:
labels.head()

0    Pastando
1    Pastando
2        EmPe
3    Pastando
4    Pastando
Name: comportamento, dtype: object

# Random Forest (Gini)

In [55]:
# sem padronização
clf = RandomForestClassifier()
scores = cross_val_score(clf, data, labels, cv=10)
[print("{}-Fold: {:.2f}".format(k, score)) for k, score in enumerate(scores, 1)]
mean = np.mean(scores)
std = np.std(scores)
print("\tAcurácia média: {:.4f}".format(mean))
print("\tDesvio Padrão: {:}".format(std))

1-Fold: 0.78
2-Fold: 0.67
3-Fold: 0.80
4-Fold: 0.86
5-Fold: 0.87
6-Fold: 0.76
7-Fold: 0.87
8-Fold: 0.73
9-Fold: 0.87
10-Fold: 0.81
	Acurácia média: 0.8024
	Desvio Padrão: 0.0648370884715836


In [58]:
# com padronização
clf = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(clf, data_normalized, labels, cv=10)
[print("{}-Fold: {:.2f}".format(k, score)) for k, score in enumerate(scores, 1)]
mean = np.mean(scores)
std = np.std(scores)
print("\tAcurácia média: {:.4f}".format(mean))
print("\tDesvio Padrão: {:}".format(std))

1-Fold: 0.77
2-Fold: 0.67
3-Fold: 0.78
4-Fold: 0.86
5-Fold: 0.87
6-Fold: 0.77
7-Fold: 0.88
8-Fold: 0.73
9-Fold: 0.86
10-Fold: 0.79
	Acurácia média: 0.7971
	Desvio Padrão: 0.06494663297071676


# Random Forest (Entropy)

In [48]:
clf = RandomForestClassifier(criterion='entropy')
scores = cross_val_score(clf, data, labels, cv=10)
[print("{}-Fold: {:.2f}".format(k, score)) for k, score in enumerate(scores, 1)]
mean = np.mean(scores)
std = np.std(scores)
print("\tAcurácia média: {:.4f}".format(mean))
print("\tDesvio Padrão: {:}".format(std))

1-Fold: 0.78
2-Fold: 0.68
3-Fold: 0.78
4-Fold: 0.86
5-Fold: 0.87
6-Fold: 0.76
7-Fold: 0.88
8-Fold: 0.73
9-Fold: 0.87
10-Fold: 0.84
	Acurácia média: 0.8056
	Desvio Padrão: 0.06599045996377269


In [ ]:
clf = RandomForestClassifier(criterion='entropy')
scores = cross_val_score(clf, data_normalized, labels, cv=10)
[print("{}-Fold: {:.2f}".format(k, score)) for k, score in enumerate(scores, 1)]
mean = np.mean(scores)
std = np.std(scores)
print("\tAcurácia média: {:.4f}".format(mean))
print("\tDesvio Padrão: {:}".format(std))

1-Fold: 0.77
2-Fold: 0.67
3-Fold: 0.78
4-Fold: 0.86
5-Fold: 0.87
6-Fold: 0.76
7-Fold: 0.88
8-Fold: 0.73
9-Fold: 0.86
10-Fold: 0.82
	Acurácia média: 0.8005
	Desvio Padrão: 0.06480470120707642


In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_features='log2', oob_score=False, n_jobs=-1, verbose=10, random_state=1, min_samples_split=10)
scores = cross_val_score(clf, data_normalized, labels, cv=10)
[print("{}-Fold: {:.2f}".format(k, score)) for k, score in enumerate(scores, 1)]
mean = np.mean(scores)
std = np.std(scores)
print("\tAcurácia média: {:.4f}".format(mean))
print("\tDesvio Padrão: {:}".format(std))  

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1901s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1743s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  86 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0617s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1687s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parall

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0706s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1738s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parall

...

[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0649s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1590s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parall

Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0692s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1736s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.9s


Using ThreadingBackend as joblib.Parallel backend instead of LokyBackend as the latter does not provide shared memory semantics.
1-Fold: 0.78
2-Fold: 0.67
3-Fold: 0.78
4-Fold: 0.85
5-Fold: 0.87
6-Fold: 0.75
7-Fold: 0.88
8-Fold: 0.73
9-Fold: 0.86
10-Fold: 0.79
	Acurácia média: 0.7965
	Desvio Padrão: 0.06482161434419732


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


In [6]:
#Bagging
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
clf = BaggingClassifier(base_estimator=SVC(),
                        n_estimators=10, random_state=0)
scores = cross_val_score(clf, data_normalized, labels, cv=10)
[print("{}-Fold: {:.2f}".format(k, score)) for k, score in enumerate(scores, 1)]
mean = np.mean(scores)
std = np.std(scores)
print("\tAcurácia média: {:.4f}".format(mean))
print("\tDesvio Padrão: {:}".format(std))

1-Fold: 0.77
2-Fold: 0.63
3-Fold: 0.70
4-Fold: 0.79
5-Fold: 0.82
6-Fold: 0.73
7-Fold: 0.85
8-Fold: 0.66
9-Fold: 0.81
10-Fold: 0.76
	Acurácia média: 0.7516
	Desvio Padrão: 0.06946885374653125
